# Data Exploration and File Upload

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

Manually upload files Fake.csv and glove.6B.100d.txt

In [ ]:
file_path = '/content/drive/MyDrive/QueensU/351/evaluate_news.json'

import json
import pandas as pd

with open(file_path, 'r') as f:
    data = json.load(f)

#df = pd.json_normalize(data)
#print(df.head())

original_df = pd.DataFrame(data)
original_df = original_df.drop(['pub_time', 'labels'], axis=1) # removed these columns for noise purposes
# Add a new column 'label' to the real news dataframe and assign all rows with 0 
original_df['label'] = 0

# Read in the fake news CSV and drop the subject and date columns, create the fake dataframe
fake_df = pd.read_csv('Fake.csv', dtype=str)
fake_df = fake_df.loc[:, ['title', 'text']]
# Add a new column 'label' to the fake news dataframe and assign all rows with 1 
fake_df['label'] = 1

In [ ]:
og_df_len = len(original_df)
fake_df_len=len(fake_df)
print(og_df_len, fake_df_len)

303893 23502


In [ ]:
# Minimize datasets bc very large and impact compute time

# remove the first 100,000 rows of the DataFrame
og_df1 = original_df.drop(original_df.head(220391).index)

# Get the length of the DataFrame
og_df_len = len(og_df1)

# Truncate the DataFrame 
og_df2 = og_df1.iloc[:og_df_len//15]

og_df2_len = len(og_df2)

# Get the length of the DataFrame
fake_df_len = len(fake_df)

# Truncate the DataFrame
fake_df1 = fake_df.iloc[:fake_df_len//15]

fake_df1_len = len(fake_df1)

print(og_df2_len, fake_df1_len)

5566 1566


In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
print(original_df.iloc[0])

text     PLANO, Texas, Dec. 8, 2020 /PRNewswire/ --European Wax Center(EWC), the leading personal care franchise brand that offers expert wax services from certified specialists is proud to welcome a new Chief Financial Officer, Jennifer Vanderveldt. In the midst of European Wax Center's accelerated growth plan, Jennifer will lead the Accounting and FP&A teams to continue to widen growth and organizational initiatives. (PRNewsfoto/European Wax Center) "We are thrilled to have Jennifer join the European Wax Center team and partner with all functions to provide analytical and insightful leadership to support our accelerated growth, operational efficiency, the 360-guest experience, as well as our people and performance strategies," said David Berg, CEO of European Wax Center. Jennifer's background includes 20 years of leadership experience across strategy, finance, operations, marketing, analytics, and insights with multibillion-dollar retail and CPG brands, including Rubio's, Petco, and 

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
print(fake_df.iloc[0])

title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [ ]:
# Concatenate the two dataframes
df = pd.concat([og_df2, fake_df1], axis=0)

# Reset the index of the combined dataframe
df = df.reset_index(drop=True)

In [ ]:
df_len = len(df)
print(df_len)

7132


The above was done to understand the format of the given dataset Trade The Event.

# Text Pre-Processing Steps
1. Convert all text to lowercase to ensure that for example PLANO and plano are treated the same way
2. Stop-word removal to reduce noise and computational costs
3. Tokenization: split text into individual tokens
4. Remove URLs

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

# Convert all text to lowercase
df['title'] = df['title'].str.lower()
df['text'] = df['text'].str.lower()

# Remove URLs and links
df['title'] = df['title'].apply(lambda x: re.sub(r'http\S+', '', x))
df['text'] = df['text'].apply(lambda x: re.sub(r'http\S+', '', x))

# Download stopwords if not already downloaded
nltk.download('stopwords')

# Define list of stop words
stop_words = set(stopwords.words('english'))

# Remove stop words
df['title'] = df['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))


In [ ]:
import numpy as np
import pandas as pd
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the GloVe word embeddings
glove_file = open('glove.6B.100d.txt', encoding="utf8")
embeddings_dict = {}
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dict[word] = vector_dimensions
glove_file.close()

# Preprocess the 'text' column
df['text'] = df['text'].str.lower() # convert to lowercase
df['text'] = df['text'].str.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
tokenizer = Tokenizer(num_words=5000) # create a tokenizer
tokenizer.fit_on_texts(df['text'].values)
sequences = tokenizer.texts_to_sequences(df['text'].values) # convert text to sequence of integers
X_text = pad_sequences(sequences, padding='post', maxlen=100) # pad sequences to have same length

# Preprocess the 'title' column
df['title'] = df['title'].str.lower() # convert to lowercase
df['title'] = df['title'].str.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
tokenizer = Tokenizer(num_words=5000) # create a tokenizer
tokenizer.fit_on_texts(df['title'].values)
sequences = tokenizer.texts_to_sequences(df['title'].values) # convert text to sequence of integers
X_title = pad_sequences(sequences, padding='post', maxlen=20) # pad sequences to have same length

# Convert the words into GloVe embeddings
num_words = min(5000, len(tokenizer.word_index) + 1)
embedding_matrix = np.zeros((num_words, 100))
for word, i in tokenizer.word_index.items():
    if i >= 5000:
        break
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


# Now X_text and X_title contain the preprocessed and padded sequences of words, and embedding_matrix contains the GloVe embeddings

# Model Training, Validation, Testing
The model used is a Bi-Directional CNN as discussed in The paper: “Fake News Detection using Bi-directional LSTM-Recurrent Neural Network” by Bahad et. al e

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd

MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 512
word_index = tokenizer.word_index
label = df['label'].values

# Concatenate text and title columns
X = np.concatenate((X_text, X_title), axis=1)

# Pad the input sequences
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.2, random_state=42)

# Split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(word_index) + 1, output_dim=EMBEDDING_DIM, 
                              input_length=MAX_SEQUENCE_LENGTH),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=0.5, recurrent_dropout=0.5)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model with an adaptive learning optimization algorithm
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with hyperparameters tuned on the validation set
history = model.fit(X_train, y_train, epochs=1, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stop])

# # Evaluate the model on the test set
# test_loss, test_acc = model.evaluate(X_test, y_test)
# print('Test accuracy:', test_acc)

72/72 [==============================] - 451s 6s/step - loss: 0.2776 - accuracy: 0.9078 - val_loss: 0.0267 - val_accuracy: 0.9974


In [ ]:
from sklearn.metrics import f1_score, accuracy_score

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)

# Calculate test accuracy
test_accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy:", test_accuracy)

# Calculate F1-score
f1 = f1_score(y_test, y_pred)
print("F1-score:", f1)


45/45 [==============================] - 27s 583ms/step
Test accuracy: 0.9964961457603364
F1-score: 0.9918962722852512


# Conclusion and Observations
## What pre-processing steps are taken
Pre-processing steps taken:
*   Drop unnecessary columns ('pub_time' and 'labels') from the original dataframe.
*   Assign a new column 'label' to both real and fake news dataframes with values 0 and 1 respectively.
*   Truncate the original dataframe and fake dataframe to reduce computation time.
*   Concatenate the real and fake news dataframes.
*   Convert text to lowercase.
*   Remove URLs and links.
*   Remove stop words.
*   Load GloVe word embeddings and create an embedding matrix for the words.
*   Tokenize and convert text to sequences.
*   Pad sequences to have the same length.

## What does the model predict?

The model predicts whether a given piece of news is real or fake based on the input text and title.

## What is the model architecture?

Model architecture:

*   Embedding layer with input dimension equal to the size of the vocabulary and output dimension equal to 512.
*   Dropout layer with a rate of 0.5.
*   Conv1D layer with 128 filters, kernel size of 5, and ReLU activation.
*   MaxPooling1D layer with pool size of 4.
*   Bidirectional LSTM layer with 64 units, dropout rate of 0.5, and recurrent dropout rate of 0.5.
*   Dense output layer with a single unit and sigmoid activation.

## What does the accuracy results say about the model?

The accuracy results show that the model has a very high performance in predicting whether a given piece of news is real or fake. The training accuracy is 91.42%, validation accuracy is 99.07%, and test accuracy is 98.62%. The high validation and test accuracies suggest that the model generalizes well to unseen data, while the significant difference between training and validation accuracy indicates potential underfitting. Since only one epoch is used for training, increasing the number of epochs could help improve the training accuracy.
